In [1]:
pip install transformers[torch] datasets sentencepiece rouge_score sacrebleu

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached portalocker-3.2.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached portalocker-3.2.0-py3-none-any.whl (22 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=25027 sha256=3f2ac4bf92f92485a4d44b13c7c1aa4c7ecf1520d7b5ad362ef8acae64315548
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score

   ------ --------------------------------- 1/6 [portalocker]
   ------------- -------------------------- 2/6 [absl-py]
   ------------- -------------------------- 2/6 [absl-py]
   -------------------- -----------------

  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print("--- GPU Information ---")
if torch.cuda.is_available():
    print("✅ CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available. PyTorch is running on CPU.")

PyTorch version: 2.5.1+cu121
--- GPU Information ---
✅ CUDA is available!
Number of GPUs: 1
GPU Name: NVIDIA RTX A5000


In [11]:
from datasets import DatasetDict, Dataset
import pandas as pd

# Load data using pandas and remove any empty rows
df = pd.read_csv('../Dataset/final_cleaned_dataset_CNN.csv').dropna().reset_index(drop=True)
raw_dataset = Dataset.from_pandas(df)

# Define the task prefixes for the model
PREFIX_ENG = "summarize English: "
PREFIX_HIN = "summarize Hindi: "

def format_dataset(batch):
    """Transforms a batch of data for the two summarization tasks."""
    inputs, targets = [], []
    
    # Create alternating examples for English and Hindi summarization
    for article, eng_summary, hin_summary in zip(
        batch['raw_news_article'],
        batch['english_summary'],
        batch['hindi_summary']
    ):
        if isinstance(article, str):
            # English Task
            inputs.append(PREFIX_ENG + article)
            targets.append(eng_summary)
            
            # Hindi Task
            inputs.append(PREFIX_HIN + article)
            targets.append(hin_summary)
            
    return {'inputs': inputs, 'targets': targets}

# Apply the formatting and create train/test splits
processed_dataset = raw_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=raw_dataset.column_names
).flatten()

train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)

final_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Verify the structure and samples
print(final_datasets)

print("\n--- English Task Sample ---")
print("Input:", final_datasets['train'][0]['inputs'])
print("Target:", final_datasets['train'][0]['targets'])

print("\n--- Hindi Task Sample ---")
print("Input:", final_datasets['train'][1]['inputs'])
print("Target:", final_datasets['train'][1]['targets'])

Map: 100%|██████████| 4919/4919 [00:00<00:00, 15757.47 examples/s]

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 8854
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 984
    })
})

--- English Task Sample ---
Input: summarize Hindi: (CNN)Canadian actor Jonathan Crombie, who co-starred in the "Anne of Green Gables" TV movies, died this week at age 48. Crombie died Wednesday from complications of a brain hemorrhage, "Anne of Green Gables" producer Kevin Sullivan said. "It's a real tragedy to see someone at age 48 go like that," he said. "I will remember him as someone who worked extremely hard to make the roles he played onscreen come to life." Based on Canadian author Lucy Maud Montgomery's children's books, "Anne of Green Gables" debuted in Canada on CBC TV in 1984 and became a cultural touchstone. The plot focused on the adventures of fiery orphan Anne Shirley, played by Megan Follows, who is sent to live on a farm in Prince Edward Island. Crombie played Gilbert Bly

Data Loading and Preparation

In [2]:
from datasets import DatasetDict, Dataset
import pandas as pd

# Load data using pandas and remove any empty rows
df = pd.read_csv('../Dataset/final_cleaned_dataset_CNN.csv').dropna().reset_index(drop=True)
raw_dataset = Dataset.from_pandas(df)

# Define the task prefixes for the model
PREFIX_ENG = "summarize English: "
PREFIX_HIN = "summarize Hindi: "

def format_dataset(batch):
    """Transforms a batch of data for the two summarization tasks."""
    inputs, targets = [], []
    
    # Create alternating examples for English and Hindi summarization
    for article, eng_summary, hin_summary in zip(
        batch['raw_news_article'],
        batch['english_summary'],
        batch['hindi_summary']
    ):
        if isinstance(article, str):
            # English Task
            inputs.append(PREFIX_ENG + article)
            targets.append(eng_summary)
            
            # Hindi Task
            inputs.append(PREFIX_HIN + article)
            targets.append(hin_summary)
            
    return {'inputs': inputs, 'targets': targets}

# Apply the formatting and create train/test splits
processed_dataset = raw_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=raw_dataset.column_names
).flatten()

train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)

final_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# --- NEW AND IMPROVED VERIFICATION ---
# Loop through the first 6 samples of the shuffled training set to see the mix of tasks.
print("--- Verifying the first 6 samples of the SHUFFLED training data ---")
for i in range(6):
    print(f"\n--- Sample {i+1} ---")
    print(final_datasets['train'][i]['inputs'])
    print("--------------------")

Map: 100%|██████████| 4919/4919 [00:00<00:00, 23037.16 examples/s]

--- Verifying the first 6 samples of the SHUFFLED training data ---

--- Sample 1 ---
summarize Hindi: (CNN)Canadian actor Jonathan Crombie, who co-starred in the "Anne of Green Gables" TV movies, died this week at age 48. Crombie died Wednesday from complications of a brain hemorrhage, "Anne of Green Gables" producer Kevin Sullivan said. "It's a real tragedy to see someone at age 48 go like that," he said. "I will remember him as someone who worked extremely hard to make the roles he played onscreen come to life." Based on Canadian author Lucy Maud Montgomery's children's books, "Anne of Green Gables" debuted in Canada on CBC TV in 1984 and became a cultural touchstone. The plot focused on the adventures of fiery orphan Anne Shirley, played by Megan Follows, who is sent to live on a farm in Prince Edward Island. Crombie played Gilbert Blythe, who evolves over time from Anne's pigtail-tugging tormentor to friend to husband. Follows and Crombie reprised the roles in the sequels "Anne of

Tokenization

In [3]:
from transformers import MT5Tokenizer

MODEL_CHECKPOINT = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(MODEL_CHECKPOINT)

MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 256

def tokenize_function(examples):
    """Converts text inputs and targets into token IDs."""
    model_inputs = tokenizer(
        examples['inputs'], 
        max_length=MAX_INPUT_LENGTH, 
        truncation=True, 
    )
    
    # Ensures labels are tokenized correctly for seq2seq tasks
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['targets'], 
            max_length=MAX_TARGET_LENGTH, 
            truncation=True,
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = final_datasets.map(tokenize_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|          | 0/8854 [00:00<?, ? examples/s]c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 984/984 [00:03<00:00, 276.40 examples/s]

Model Training

In [4]:
import numpy as np
import evaluate
from transformers import (
    MT5ForConditionalGeneration, 
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

model = MT5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
rouge_metric = evaluate.load("rouge")

BATCH_SIZE = 4
MODEL_NAME = "mt5-base-cnn-summarizer-en-hi"

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_NAME,
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    fp16=False, # <--- THIS IS THE FIX
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    """Calculates ROUGE scores for evaluation."""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    return {k: round(v * 100, 4) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()
trainer.save_model(f"{MODEL_NAME}/final_model")

c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 50/6642 [01:16<2:44:49,  1.50s/it]

{'loss': 18.8898, 'grad_norm': 25843.544921875, 'learning_rate': 5e-06, 'epoch': 0.02}


  2%|▏         | 100/6642 [02:53<5:24:53,  2.98s/it]

{'loss': 17.3691, 'grad_norm': 24710.8671875, 'learning_rate': 1e-05, 'epoch': 0.05}


  2%|▏         | 150/6642 [04:29<2:45:05,  1.53s/it]

...

{'loss': 2.5136, 'grad_norm': 2.202885866165161, 'learning_rate': 3.4939759036144585e-05, 'epoch': 1.06}


 36%|███▌      | 2400/6642 [59:09<1:14:42,  1.06s/it]

{'loss': 2.5112, 'grad_norm': 2.123568296432495, 'learning_rate': 3.453272549658092e-05, 'epoch': 1.08}


 36%|███▌      | 2403/6642 [59:12<1:12:46,  1.03s/it]

KeyboardInterrupt: 

In [2]:
import numpy as np
import evaluate
from transformers import (
    MT5ForConditionalGeneration, 
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

model = MT5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
rouge_metric = evaluate.load("rouge")

BATCH_SIZE = 4
MODEL_NAME = "mt5-base-cnn-summarizer-en-hi-v4"

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_NAME,
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    fp16=False, # <--- THIS IS THE FIX
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_pred):
    """Calculates ROUGE scores for evaluation."""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    return {k: round(v * 100, 4) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()
trainer.save_model(f"{MODEL_NAME}/final_model")

c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/6642 [00:00<?, ?it/s]

{'loss': 18.8898, 'grad_norm': 25843.544921875, 'learning_rate': 5e-06, 'epoch': 0.02}
{'loss': 17.3691, 'grad_norm': 24710.8671875, 'learning_rate': 1e-05, 'epoch': 0.05}
{'loss': 15.1896, 'grad_norm': 1253.855712890625, 'learning_rate': 1.5e-05, 'epoch': 0.07}
{'loss': 11.4705, 'grad_norm': 1275.7470703125, 'learning_rate': 2e-05, 'epoch': 0.09}
{'loss': 7.257, 'grad_norm': 19.44481086730957, 'learning_rate': 2.5e-05, 'epoch': 0.11}
{'loss': 4.3326, 'grad_norm': 11.529706954956055, 'learning_rate': 3e-05, 'epoch': 0.14}
{'loss': 3.6727, 'grad_norm': 3.133930206298828, 'learning_rate': 3.5e-05, 'epoch': 0.16}
{'loss': 3.3826, 'grad_norm': 3.2587127685546875, 'learning_rate': 4e-05, 'epoch': 0.18}
{'loss': 3.4258, 'grad_norm': 2.94587779045105, 'learning_rate': 4.5e-05, 'epoch': 0.2}
{'loss': 3.2653, 'grad_norm': 2.916131019592285, 'learning_rate': 5e-05, 'epoch': 0.23}
{'loss': 3.1574, 'grad_norm': 2.276667833328247, 'learning_rate': 4.959296646043634e-05, 'epoch': 0.25}
{'loss': 3.14

c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/246 [00:00<?, ?it/s]

...

  0%|          | 0/246 [00:00<?, ?it/s]

2025-09-22 09:16:36,932 [INFO] - Using default tokenizer.


{'eval_loss': 1.8242502212524414, 'eval_rouge1': 11.8785, 'eval_rouge2': 4.6153, 'eval_rougeL': 10.457, 'eval_rougeLsum': 11.9025, 'eval_runtime': 204.8678, 'eval_samples_per_second': 4.803, 'eval_steps_per_second': 1.201, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'train_runtime': 9039.8248, 'train_samples_per_second': 2.938, 'train_steps_per_second': 0.735, 'train_loss': 2.9490119522862033, 'epoch': 3.0}


In [ ]:
import logging
import pandas as pd
import numpy as np
import evaluate
from datetime import datetime
from datasets import DatasetDict, Dataset
from transformers import (
    MT5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    MT5Tokenizer,
    pipeline
)

# Configure logger to save to a unique file and print to console
log_filename = f"training_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

logging.info("--- Starting New Training Run ---")

try:
    logging.info("Step 2: Starting data loading and preparation.")
    df = pd.read_csv('../Dataset/final_cleaned_dataset_CNN.csv').dropna().reset_index(drop=True)
    raw_dataset = Dataset.from_pandas(df)

    PREFIX_ENG = "summarize English: "
    PREFIX_HIN = "summarize Hindi: "

    def format_dataset(batch):
        inputs, targets = [], []
        for article, eng_summary, hin_summary in zip(
            batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
        ):
            if isinstance(article, str):
                inputs.append(PREFIX_ENG + article)
                targets.append(eng_summary)
                inputs.append(PREFIX_HIN + article)
                targets.append(hin_summary)
        return {'inputs': inputs, 'targets': targets}

    processed_dataset = raw_dataset.map(
        format_dataset, batched=True, remove_columns=raw_dataset.column_names
    ).flatten()

    train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
    final_datasets = DatasetDict({
        'train': train_test_split['train'], 'test': train_test_split['test']
    })
    logging.info(f"Data prepared successfully. Training samples: {len(final_datasets['train'])}, Test samples: {len(final_datasets['test'])}")

    logging.info("Step 3: Starting tokenization.")
    MODEL_CHECKPOINT = "google/mt5-base"
    tokenizer = MT5Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    MAX_INPUT_LENGTH, MAX_TARGET_LENGTH = 1024, 256

    def tokenize_function(examples):
        model_inputs = tokenizer(examples['inputs'], max_length=MAX_INPUT_LENGTH, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples['targets'], max_length=MAX_TARGET_LENGTH, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = final_datasets.map(tokenize_function, batched=True)
    logging.info("Tokenization complete.")

    logging.info("Step 4: Initializing Trainer and preparing for model training.")
    model = MT5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
    rouge_metric = evaluate.load("rouge")
    BATCH_SIZE = 4
    MODEL_NAME = "mt5-base-cnn-summarizer-en-hi"

    training_args = Seq2SeqTrainingArguments(
        output_dir=MODEL_NAME,
        num_train_epochs=3,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f"{MODEL_NAME}/logs",
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch",
        predict_with_generate=True,
        fp16=False,
        load_best_model_at_end=True,
        metric_for_best_model="rouge2",
        generation_max_length=256
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
        decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
        result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        return {k: round(v * 100, 4) for k, v in result.items()}

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    logging.info("Starting model training...")
    trainer.train()
    logging.info("Training finished successfully.")
    trainer.save_model(f"{MODEL_NAME}/final_model")
    logging.info(f"Model saved to {MODEL_NAME}/final_model")

    logging.info("Step 5: Performing inference with the trained model.")
    model_path = f"{MODEL_NAME}/final_model"
    summarizer = pipeline("summarization", model=model_path, tokenizer=model_path)
    
    article_text = """
    The Indian Space Research Organisation (ISRO) successfully launched its ambitious Mars Orbiter Mission, also known as Mangalyaan, making India the first nation to succeed on its maiden attempt to reach Mars. The low-cost mission, which cost only $74 million, was designed to study the Martian atmosphere and surface. The spacecraft orbited Mars for several years, sending back valuable data and images, far exceeding its expected mission life. The success of Mangalyaan was a major milestone for India's space program, demonstrating its capability to execute complex interplanetary missions and cementing its position as a major player in space exploration.
    """

    english_summary = summarizer(PREFIX_ENG + article_text, max_length=100)
    logging.info(f"--- English Summary ---\n{english_summary[0]['summary_text']}")

    hindi_summary = summarizer(PREFIX_HIN + article_text, max_length=120)
    logging.info(f"--- Hindi Summary ---\n{hindi_summary[0]['summary_text']}")

    logging.info("--- Run Completed Successfully ---")

except Exception as e:
    logging.error(f"An unexpected error occurred during the run: {e}", exc_info=True)
    logging.error("--- Run Failed ---")

Map: 100%|██████████| 4919/4919 [00:00<00:00, 21255.17 examples/s]
c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 984/984 [00:03<00:00, 272.22 examples/s]
                                                     
 36%|███▌      | 2403/6642 [1:32:22<1:12:46,  1.03s/it]

{'loss': 18.8898, 'grad_norm': 25843.544921875, 'learning_rate': 5e-06, 'epoch': 0.02}


                                                       
 36%|███▌      | 2403/6642 [1:59:13<1:12:46,  1.03s/it]

{'loss': 17.3691, 'grad_norm': 24710.8671875, 'learning_rate': 1e-05, 'epoch': 0.05}


                                                       
 36%|███▌      | 2403/6642 [2:30:24<1:12:46,  1.03s/it]

{'loss': 15.1896, 'grad_norm': 1253.855712890625, 'learning_rate': 1.5e-05, 'epoch': 0.07}


In [ ]:
import logging
import pandas as pd
import numpy as np
import evaluate
from datetime import datetime
from datasets import DatasetDict, Dataset
from transformers import (
    MT5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    MT5Tokenizer,
    pipeline
)

# Configure logger
log_filename = f"training_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

logging.info("--- Starting New Training Run with GPU ---")

try:
    logging.info("Step 2: Starting data loading and preparation.")
    df = pd.read_csv('../Dataset/final_cleaned_dataset_CNN.csv', engine='python', on_bad_lines='skip')
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    raw_dataset = Dataset.from_pandas(df)

    PREFIX_ENG = "summarize English: "
    PREFIX_HIN = "summarize Hindi: "

    def format_dataset(batch):
        inputs, targets = [], []
        for article, eng_summary, hin_summary in zip(
            batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
        ):
            if isinstance(article, str):
                inputs.append(PREFIX_ENG + article)
                targets.append(eng_summary)
                inputs.append(PREFIX_HIN + article)
                targets.append(hin_summary)
        return {'inputs': inputs, 'targets': targets}

    processed_dataset = raw_dataset.map(
        format_dataset, batched=True, remove_columns=raw_dataset.column_names
    ).flatten()

    train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
    final_datasets = DatasetDict({
        'train': train_test_split['train'], 'test': train_test_split['test']
    })
    logging.info(f"Data prepared successfully. Samples: {len(final_datasets['train'])} train, {len(final_datasets['test'])} test.")

    logging.info("Step 3: Starting tokenization.")
    MODEL_CHECKPOINT = "google/mt5-base"
    tokenizer = MT5Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    MAX_INPUT_LENGTH, MAX_TARGET_LENGTH = 1024, 256

    def tokenize_function(examples):
        model_inputs = tokenizer(examples['inputs'], max_length=MAX_INPUT_LENGTH, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples['targets'], max_length=MAX_TARGET_LENGTH, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = final_datasets.map(tokenize_function, batched=True)
    logging.info("Tokenization complete.")

    logging.info("Step 4: Initializing Trainer.")
    model = MT5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
    rouge_metric = evaluate.load("rouge")
    BATCH_SIZE = 4
    MODEL_NAME = "mt5-base-cnn-summarizer-en-hi"

    training_args = Seq2SeqTrainingArguments(
        output_dir=MODEL_NAME,
        num_train_epochs=3,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f"{MODEL_NAME}/logs",
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch",
        predict_with_generate=True,
        fp16=True,  # Re-enabled for RTX A5000 performance
        load_best_model_at_end=True,
        metric_for_best_model="rouge2",
        generation_max_length=256
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
        decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
        result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        return {k: round(v * 100, 4) for k, v in result.items}

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    logging.info("Starting model training on GPU...")
    trainer.train()
    logging.info("Training finished successfully.")
    trainer.save_model(f"{MODEL_NAME}/final_model")
    logging.info(f"Model saved to {MODEL_NAME}/final_model")

    logging.info("--- Run Completed Successfully ---")

except Exception as e:
    logging.error(f"An unexpected error occurred: {e}", exc_info=True)
    logging.error("--- Run Failed ---")

2025-09-22 05:18:51,366 [INFO] - --- Starting New Training Run with GPU ---
2025-09-22 05:18:51,366 [INFO] - Step 2: Starting data loading and preparation.


Map:   0%|          | 0/4919 [00:00<?, ? examples/s]

2025-09-22 05:18:52,218 [INFO] - Data prepared successfully. Samples: 8854 train, 984 test.
2025-09-22 05:18:52,218 [INFO] - Step 3: Starting tokenization.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/8854 [00:00<?, ? examples/s]

c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/984 [00:00<?, ? examples/s]

2025-09-22 05:19:31,647 [INFO] - Tokenization complete.
2025-09-22 05:19:31,647 [INFO] - Step 4: Initializing Trainer.
2025-09-22 05:19:45,913 [INFO] - Starting model training on GPU...


  0%|          | 0/6642 [00:00<?, ?it/s]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.02}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.05}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.07}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.11}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.14}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.16}


KeyboardInterrupt: 

In [ ]:
import logging
import pandas as pd
import numpy as np
import evaluate
from datetime import datetime
from datasets import DatasetDict, Dataset
from transformers import (
    MT5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    MT5Tokenizer,
    pipeline
)

# Configure a logger to save progress and errors to a timestamped file
log_filename = f"training_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

logging.info("--- Starting New Training Run with GPU ---")

try:
    logging.info("Starting data loading and preparation.")
    # Robustly read the CSV, skipping any malformed lines
    df = pd.read_csv('../Dataset/final_cleaned_dataset_CNN.csv', engine='python', on_bad_lines='skip')
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    raw_dataset = Dataset.from_pandas(df)

    PREFIX_ENG = "summarize English: "
    PREFIX_HIN = "summarize Hindi: "

    def format_dataset(batch):
        inputs, targets = [], []
        for article, eng_summary, hin_summary in zip(
            batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
        ):
            if isinstance(article, str):
                inputs.append(PREFIX_ENG + article)
                targets.append(eng_summary)
                inputs.append(PREFIX_HIN + article)
                targets.append(hin_summary)
        return {'inputs': inputs, 'targets': targets}

    processed_dataset = raw_dataset.map(
        format_dataset, batched=True, remove_columns=raw_dataset.column_names
    ).flatten()

    train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
    final_datasets = DatasetDict({
        'train': train_test_split['train'], 'test': train_test_split['test']
    })
    logging.info(f"Data prepared. Samples: {len(final_datasets['train'])} train, {len(final_datasets['test'])} test.")

    logging.info("Starting tokenization.")
    MODEL_CHECKPOINT = "google/mt5-base"
    tokenizer = MT5Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    MAX_INPUT_LENGTH, MAX_TARGET_LENGTH = 1024, 256

    def tokenize_function(examples):
        model_inputs = tokenizer(examples['inputs'], max_length=MAX_INPUT_LENGTH, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples['targets'], max_length=MAX_TARGET_LENGTH, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = final_datasets.map(tokenize_function, batched=True)
    logging.info("Tokenization complete.")

    logging.info("Initializing Trainer.")
    model = MT5ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
    rouge_metric = evaluate.load("rouge")
    BATCH_SIZE = 4
    MODEL_NAME = "mt5-base-cnn-summarizer-en-hi_v3"

    training_args = Seq2SeqTrainingArguments(
        output_dir=MODEL_NAME,
        num_train_epochs=3,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f"{MODEL_NAME}/logs",
        logging_steps=50,
        evaluation_strategy="epoch", # Use "eval_strategy" in newer library versions
        save_strategy="epoch",
        predict_with_generate=True,
        fp16=False, # Set to False for maximum stability
        load_best_model_at_end=True,
        metric_for_best_model="rouge2",
        generation_max_length=256,
        adam_epsilon=1e-8 # Added for optimizer stability
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        # Clean up potentially invalid token IDs before decoding
        predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
        decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
        result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        return {k: round(v * 100, 4) for k, v in result.items()}

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    logging.info("Starting model training on GPU...")
    trainer.train()
    logging.info("Training finished successfully.")
    trainer.save_model(f"{MODEL_NAME}/final_model")
    logging.info(f"Model saved to {MODEL_NAME}/final_model")

    logging.info("Performing inference with the trained model.")
    model_path = f"{MODEL_NAME}/final_model"
    summarizer = pipeline("summarization", model=model_path, tokenizer=model_path)
    
    article_text = """
    The Indian Space Research Organisation (ISRO) successfully launched its ambitious Mars Orbiter Mission, also known as Mangalyaan, making India the first nation to succeed on its maiden attempt to reach Mars. The low-cost mission, which cost only $74 million, was designed to study the Martian atmosphere and surface. The spacecraft orbited Mars for several years, sending back valuable data and images, far exceeding its expected mission life. The success of Mangalyaan was a major milestone for India's space program, demonstrating its capability to execute complex interplanetary missions and cementing its position as a major player in space exploration.
    """

    english_summary = summarizer(PREFIX_ENG + article_text, max_length=100)
    logging.info(f"--- English Summary ---\n{english_summary[0]['summary_text']}")

    hindi_summary = summarizer(PREFIX_HIN + article_text, max_length=120)
    logging.info(f"--- Hindi Summary ---\n{hindi_summary[0]['summary_text']}")

    logging.info("--- Run Completed Successfully ---")

except Exception as e:
    logging.error(f"An unexpected error occurred: {e}", exc_info=True)
    logging.error("--- Run Failed ---")

2025-09-22 13:56:36,347 [INFO] - --- Starting New Training Run with GPU ---
2025-09-22 13:56:36,347 [INFO] - Starting data loading and preparation.


Map:   0%|          | 0/4919 [00:00<?, ? examples/s]

2025-09-22 13:56:37,191 [INFO] - Data prepared. Samples: 8854 train, 984 test.
2025-09-22 13:56:37,191 [INFO] - Starting tokenization.


Map:   0%|          | 0/8854 [00:00<?, ? examples/s]

c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/984 [00:00<?, ? examples/s]

2025-09-22 13:57:16,484 [INFO] - Tokenization complete.
2025-09-22 13:57:16,484 [INFO] - Initializing Trainer.
c:\Users\admin\anaconda3\envs\news_summarizer\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2025-09-22 13:57:27,517 [INFO] - Starting model training on GPU...


  0%|          | 0/6642 [00:00<?, ?it/s]

{'loss': 18.8898, 'grad_norm': 25843.544921875, 'learning_rate': 5e-06, 'epoch': 0.02}


Inference of the MOdel


In [4]:
from transformers import pipeline

model_path = f"{MODEL_NAME}/final_model"
summarizer = pipeline("summarization", model=model_path, tokenizer=model_path)

article_text = """
‘Insult To Every Mother’: PM Modi Tears Into Congress-RJD For Abusing His Late Mother
"It is well-known that someone who has disowned his own sister and stabbed his own brother in the back cannot be expected to respect any woman. If he had education and values, he would never have resorted to such cheap tactics just before the Devi Paksha," he continued.He went on to declare that the women in poll-bound Bihar would "give a fitting reply to Tejashwi, who stood on the stage and got his own workers to insult the late mother of Prime Minister ji.""The people of Bihar are hurt and outraged by the insult to Prime Minister Modi ji’s late mother," he said.The slugfest over alleged abuses flared up again after Bihar deputy CM Samrat Choudhary shared a video of Tejashwi's rally accusing RJD leader of encouraging party workers to hurl abuses. This came only weeks after a similar row during Rahul Gandhi’s Darbhanga rally."Tejashwi Yadav once again had Modi's deceased mother abused. He has once again shattered Bihar's culture. The more RJD workers hurled abuses at the rally, the more Tejashwi Yadav encouraged them. The mothers and sisters of Bihar will surely take account of this hooligan mentality and abuse," he said.RJD defended itself saying that the video was "fabricated". “Leader of opposition Tejashwi Prasad Yadav ji is delivering a speech today at Mahua during the Bihar Adhikar Yatra. The speech is available on my Facebook page — you can listen to it. In it, no worker or any person has abused or used foul language for the Hon’ble Prime Minister,” party leader Mukesh Raushan said."""

# Generate English Summary using the English prefix
english_summary = summarizer(PREFIX_ENG + article_text, max_length=100)
print("--- English Summary ---")
print(english_summary[0]['summary_text'])

# Generate Hindi Summary using the Hindi prefix
hindi_summary = summarizer(PREFIX_HIN + article_text, max_length=120)
print("\n--- Hindi Summary ---")
print(hindi_summary[0]['summary_text'])

--- English Summary ---
Bihar PM Modi threatened Congress-RJD for Abusing His Late Mother, asserting that someone who disowned his sister and stabbed his own brother in the back cannot be expected to respect any woman. He asserted that Bihar would "give a fitting reply" to Tejashwi, who stood on stage and encouraged party workers to hurl abuses. Bihar deputy CM Samrat Choudhar

--- Hindi Summary ---
मैनेजर PM मोदी ने कांग्रेस-RJD के अध्यक्ष टीजाशवी पीरादा याददा के विरोध में पार्टी के अध्यक्ष तेजाशवी पीरादा याददा के विरोध में विरोध प्रदर्शन किया, जिसमें वह अपने पिता और अपनी बेटी को गालियों से संबोधित करने के लिए विरोध प्रदर्शन कर रहे थे। यह घटना Bihar के मुख्य मंत्री रमेश गंदेदी ने बाद में बाद में
